In [10]:
# import libraries
# Thanks https://forum.freecodecamp.org/t/machine-learning-with-python-projects-neural-network-sms-text-classifier/752803/3 for library fix
!pip uninstall -y tf-nightly keras-nightly protobuf
!pip install tensorflow==2.20.0
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

# Thanks https://www.tensorflow.org/text/tutorials/text_classification_rnn for example of text classification, this was a very helpful starting point
# (This is referenced as "TF Text Classification example" in later cells)
BUFFER_SIZE = 10000
BATCH_SIZE = 32
VOCAB_SIZE = 1000

Found existing installation: protobuf 6.32.1
Uninstalling protobuf-6.32.1:
  Successfully uninstalled protobuf-6.32.1
  Using cached protobuf-6.32.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
Using cached protobuf-6.32.1-cp39-abi3-manylinux2014_x86_64.whl (322 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.32.1 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.20.0 which is incompatible.
grpcio-status 1.71.2 requires proto

2.20.0


In [11]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-10-07 17:06:14--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2025-10-07 17:06:14 (10.4 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2025-10-07 17:06:15--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

In [12]:
# Import data
# Thanks https://stackoverflow.com/a/9652858 for sep parameter
train_dataset = pd.read_csv(train_file_path, sep='\t', header=None)
test_dataset = pd.read_csv(test_file_path, sep='\t', header=None)

# Thanks https://www.geeksforgeeks.org/pandas/add-column-names-to-dataframe-in-pandas/ for adding column names
train_dataset.columns = ['type', 'message']
test_dataset.columns = ['type', 'message']

# Convert ham/spam classification to numeric
train_dataset['type'] = train_dataset['type'].astype('category')
categories = train_dataset.select_dtypes(['category']).columns
train_dataset[categories] = train_dataset[categories].apply(lambda x: x.cat.codes)

test_dataset['type'] = test_dataset['type'].astype('category')
categories = test_dataset.select_dtypes(['category']).columns
test_dataset[categories] = test_dataset[categories].apply(lambda x: x.cat.codes)

# Pop ham/spam labels, this is the output for determining if msg is spam or not
train_labels = train_dataset.pop('type')
test_labels = test_dataset.pop('type')

In [13]:
# Convert pandas DataFrames to TensorFlow Datasets
# This is needed so that the datasets will work in model.fit, thanks https://www.tensorflow.org/tutorials/load_data/pandas_dataframe#with_tfdata
tf_train_dataset = tf.data.Dataset.from_tensor_slices((train_dataset['message'], train_labels))
tf_test_dataset = tf.data.Dataset.from_tensor_slices((test_dataset['message'], test_labels))

# Shuffle the data, this is needed for training
# Thanks TF Text Classification example for method
tf_train_dataset = tf_train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
tf_test_dataset = tf_test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [14]:
# Encode text from words to numeric
# Thanks TF Text Classification example for method
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(tf_train_dataset.map(lambda text, label: text))

In [15]:
# Create the model
# The encoder and embedding layers are needed as the words are encoded, this is from the TF Text Classification example
# After that, use LSTM and Dense layers
model = tf.keras.Sequential([
    encoder, tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True
    ),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

# Fit model to data
model.fit(
    tf_train_dataset,
    epochs=10
)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.8972 - loss: 0.3668
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.9812 - loss: 0.0906
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.9424 - loss: 0.2585
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.9934 - loss: 0.0597
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.9953 - loss: 0.0482
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.9973 - loss: 0.0331
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.9821 - loss: 0.0904
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.9973 - loss: 0.0257
Epoch 9/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9959 - loss: 0.0576
Epoch 10/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.9976 - loss: 0.0349


In [16]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Input needs to be a tensor, thanks https://www.geeksforgeeks.org/python/python-tensorflow-constant/ for tf.constant
  prediction = model.predict(tf.constant([pred_text]))

  # Get direct value
  value = prediction[0][0].item()

  return [value, "ham" if value < 0 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
[-0.5609880089759827, 'ham']


In [17]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(f"Message: {msg}, Prediction: {prediction}, Expected: {ans}")
    # Access the label from the list returned by predict_message
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Message: how are you doing today, Prediction: [-0.5609880089759827, 'ham'], Expected: ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Message: sale today! to stop texts call 98912460324, Prediction: [0.8507645726203918, 'spam'], Expected: spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Message: i dont want to go. can we try it a different day? available sat, Prediction: [-0.608105480670929, 'ham'], Expected: ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Message: our new mobile video service is live. just install on your phone to start watching., Prediction: [3.0606138706207275, 'spam'], Expected: spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Message: you have won £1000 cash! call to claim your prize., Prediction: [3.5555672645568848, 'spam'], Expected: spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Message: i'll bring it tomorrow. don't forget the milk., Prediction: [-0.5961716771125793, 'ham'], Expected: ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Message: wow, is your a